# ASP工具箱教程3：触发器的跨图移植
### 攥写：<font color="grey">[babycat](https://github.com/babycat262)</font>        技术支持：<font color="grey">[Alian713](https://github.com/Divy1211)</font>

    有时候，我们在场景中设置了一些通用功能的触发，不仅希望能够在本场景中使用，也希望移植到其他场景里。就可以利用ASP实现触发器的跨图移植。
在之前的例子中我们给场景 **Sanatorium.aoe2scenario**导入了一些常用的的XS函数。现在我想把另一个场景 **shu_tribes.aoe2scenario** 中与日历和季节交替有关的触发器移植到场景 **Sanatorium.aoe2scenario**中，应该怎么做呢？

在移植之前，我们先来看看**the exiled prince(nultui).aoe2scenario**中与日历机制有关的触发器详情。在地图编辑器打开**the exiled prince(nultui).aoe2scenario**找到日历相关触发器如下图所示：

<img src="./images/calendar.png" width=360 height=400 align="left">
<img src="./images/calendar_xs.png" width=700 height=300 align="right">

从上左图可以看出，日历+季节系统 总共用了11个触发器；右图中是日历更新机制的XS调用`calendar_system();`，以2s的频率循环调用以更新日历时间；每次调用，日历中的最小时间单位“天”（ 1“天” = 2s ）增加1 

**日历更新机制**（外置XS）的完整实现如下：

```c++
// 日历更新周期 T = 2s
const int TV_year = 11;    // 存储year值的触发器变量
const int TV_month = 12;    // 存储month值的触发器变量
const int TV_day = 13;    // 存储day值的触发器变量
const int TV_yday = 14;    // 存储yday值的触发器变量

// 根据year值确定是否闰年
int isLeap(int year=1000) {if((year%400 == 0) || (year%4 == 0) && (year%100 != 0)){return (1);} return (0);}

void calendar_system()
{
    static int runs = 0;  // 日历累计运行次数
    static int leap = 0;  // 闰年标识
    //日历初始信息
    static int year = -240;  // 始值年份
    static int month= 02;  // 始值月份
    static int day  = 04;  // 月内天数
    static int yday = 35;  // 一年内第y天
    if(isLeap(year)==1 && runs==0) {leap = 1;}  //确定初始年份的闰年标识
    day++; yday++;    // 更新日期
    if(yday==32 || yday==60+leap || yday==91+leap 
       || yday==121+leap || yday==152+leap || yday==182+leap 
       || yday==213+leap || yday==244+leap || yday==274+leap 
       || yday==305+leap || yday==335+leap ) { month++; day=1; }
    else if(yday > 365+leap)
    {
        year++;
        month=1;
        day=1;
        yday=1;
        leap = isLeap(year);  //判断新的一年是否闰年
    }
    else {}
    // 同步触发器变量
    xsSetTriggerVariable(TV_year, year);
    xsSetTriggerVariable(TV_month, month);
    xsSetTriggerVariable(TV_day, day);
    xsSetTriggerVariable(TV_yday, yday);
    runs++;
}
```

#### 用ASP读取场景文件

In [1]:
from AoE2ScenarioParser.scenarios.aoe2_de_scenario import AoE2DEScenario
# 从utils中导入工具函数，实现对文件脚本的读取
from utils import load_const, load_function

In [2]:
# 设置玩家基本信息
infos = {
    "user": "babycat",
    "game_id": 76561198386517457,
}
# 场景基本信息
scx1 = "the exiled prince(multi)"  # TODO change to your scx1
scx2 = "shu_tribes_demo"    # 要迁移触发器的目标场景
migrate_scx = "shu_tribes_demo_Trans"    # 触发器迁移后的输出场景
# 移植触发的源场景，目标场景，以及移植后地图的保存路径
scx1_path = f"C:/Users/{infos['user']}/Games/Age of Empires 2 DE/{infos['game_id']}/resources/_common/scenario/{scx1}.aoe2scenario"
scx2_path = f"C:/Users/{infos['user']}/Games/Age of Empires 2 DE/{infos['game_id']}/resources/_common/scenario/{scx2}.aoe2scenario"
des_path  = f"C:/Users/{infos['user']}/Games/Age of Empires 2 DE/{infos['game_id']}/resources/_common/scenario/{migrate_scx}.aoe2scenario"

In [3]:
# 用ASP加载场景
scenario1 = AoE2DEScenario.from_file(scx1_path)
scenario2 = AoE2DEScenario.from_file(scx2_path)
# 触发管理器
trigger_mgr1 = scenario1.trigger_manager
trigger_mgr2 = scenario2.trigger_manager
# 触发列表中的触发器数量
print(f"源场景{scx1}中的触发器数量：", trigger_mgr1.triggers.__len__())
print(f"目标场景{scx2}中的触发器数量：", trigger_mgr2.triggers.__len__())


[12:21:52] Reading file: 'C:/Users/babycat/Games/Age of Empires 2 DE/76561198386517457/resources/_common/scenario/the exiled prince(multi).aoe2scenario'
[12:21:52] Reading scenario file finished successfully.

############### Attributes ###############
>>> Game version: 'DE'
>>> Scenario version: 1.53
>>> Scenario variant: 'Age of Empires 2'
##########################################

[12:21:52] Loading scenario structure finished successfully.
[12:21:52] Parsing scenario file...
	✔ FileHeadereHeader data...
	✔ DataHeaderaHeader data...
	✔ Messagesessages data...
	✔ Cinematicsematics data...
	✔ BackgroundImagendImage data...
	✔ PlayerDataTwoDataTwo data...
	✔ GlobalVictoryVictory data...
	✔ Diplomacyplomacy data...
	✔ OptionsOptions data...
	✔ Maping Map data...
	✔ Unitsg Units data...
	✔ Triggersriggers data...
	✔ Filesg Files data...
[12:21:58] Parsing scenario file finished successfully.

[12:21:58] Setting up managers ...
	✔ MessageManagereManager...
	✔ PlayerManagerrManager...
	✔

#### 查看源场景中跟日历相关的触发器信息，并将他们筛选出来

In [4]:
# 日历相关的触发器名称列表
calendar_trigger_names = ["***** 日历+季节系统 *****", 
                          "[S] 显示日历", 
                          "春季 UI", 
                          "夏季 UI", 
                          "秋季 UI", 
                          "冬季 UI", 
                          "季节交替（春季）Entrance", 
                          "季节交替（夏季）", 
                          "季节交替（秋季）", 
                          "季节交替（冬季）", 
                          "日历更新系统（2s/day）"]
copy_indices = []
# 循环遍历源场景的触发列表筛选出日历相关触发器
for t1 in trigger_mgr1.triggers:
    if t1.name in calendar_trigger_names:
        copy_indices.append(t1.trigger_id)

print("Need copy indices of trigger: ", copy_indices)

Need copy indices of trigger:  [124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134]


从以上输出可以看到，日历相关的触发器范围为 [124, 134]。需要将这些触发器移植到目标场景 **shu_tribes_demo.aoe2scenario**
### 开始移植

In [5]:
# 指定要移植的触发器的起始触发ID以及结束触发ID
pos_start = 124
pos_end = 134
# 筛选出要移植的触发器
target_triggers = trigger_mgr1.triggers[pos_start : pos_end+1]
target_triggers

[Trigger: 	{
 		_instance_number_history: 	[]
 		_uuid: 3ea5287b-48c2-484c-ba95-162b5feb996c
 		name: ***** 日历+季节系统 *****
 		description: 
 		description_stid: 0
 		display_as_objective: 0
 		short_description: 
 		short_description_stid: 0
 		display_on_screen: 0
 		description_order: 0
 		enabled: 0
 		looping: 0
 		header: 0
 		mute_objectives: 0
 		conditions: []
 		condition_order: 	[]
 		effects: [Effect: 	{
 			_instance_number_history: 	[]
 			_uuid: 3ea5287b-48c2-484c-ba95-162b5feb996c
 			effect_type: 8
 			ai_script_goal: -1
 			quantity: -1
 			tribute_list: -1
 			diplomacy: -1
 			legacy_location_object_reference: -1
 			object_list_unit_id: -1
 			source_player: -1
 			target_player: -1
 			technology: -1
 			string_id: -1
 			display_time: -1
 			trigger_id: 134
 			location_x: -1
 			location_y: -1
 			location_object_reference: -1
 			area_x1: -1
 			area_y1: -1
 			area_x2: -1
 			area_y2: -1
 			object_group: -1
 			object_type: -1
 			instruction_panel_position: -1

In [6]:
# 指定这些触发要导入目标场景中触发列表的位置， -1表示从尾部导入触发
insert_idx = -1
trigger_mgr2.import_triggers(triggers=target_triggers, index=insert_idx)
# 按照显示序重排触发器
trigger_mgr2.reorder_triggers(trigger_mgr2.trigger_display_order)

In [7]:
# 将经过触发移植的目标场景保存
scenario2.write_to_file(des_path)


[12:26:44] Reconstructing sections and structs from managers...
	✔ MessageManagerssageManager...
	✔ PlayerManagerlayerManager...
	✔ MapManagerg MapManager...
	✔ UnitManager UnitManager...
	✔ TriggerManageriggerManager...
	✔ XsManagerng XsManager...
	✔ OptionManagerptionManager...
[12:26:46] Reconstruction finished successfully.

[12:26:46] File writing from structure started...
	✔ FileHeaderg FileHeader...
	✔ DataHeaderg DataHeader...
	✔ Messagesing Messages...
	✔ Cinematicsg Cinematics...
	✔ BackgroundImagekgroundImage...
	✔ PlayerDataTwolayerDataTwo...
	✔ GlobalVictorylobalVictory...
	✔ Diplomacyng Diplomacy...
	✔ Optionsting Options...
	✔ Maptructing Map...
	✔ Unitsucting Units...
	✔ Triggersing Triggers...
	✔ Filesucting Files...
[12:26:47] File writing finished successfully.
[12:26:47] File successfully written to: 'outputs/shu_tribes_demo_Trans.aoe2scenario'
[12:26:47] Execution time from scenario read: 294.87s


<img src="images/saved_copyTS.png" width=700 height=350 align="left">
<img src="images/calendar_cpoy.png" width=360 height=420 align="right">

#### 场景保存成功后，在地图编辑器中打开场景**shu_tribes_demo_Trans.aoe2scenario**, 可以看到日历机制相关的触发已经成功移植到目标场景中。